In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("Wallet id - Sheet1.csv")

# Simulate Compound-like activity data
np.random.seed(42)
df['total_supplied_usd'] = np.random.uniform(1000, 10000, len(df))
df['total_borrowed_usd'] = np.random.uniform(500, 9000, len(df))
df['liquidation_count'] = np.random.randint(0, 4, len(df))
df['total_repaid_usd'] = df['total_borrowed_usd'] * np.random.uniform(0.4, 1.1, len(df))
df['activity_count'] = np.random.randint(5, 50, len(df))
df['days_since_last_txn'] = np.random.randint(0, 365, len(df))

# Compute derived features
df['borrow_to_supply_ratio'] = df['total_borrowed_usd'] / df['total_supplied_usd']
df['repayment_ratio'] = df['total_repaid_usd'] / df['total_borrowed_usd']
df['utilization_rate'] = df['total_borrowed_usd'] / (df['total_supplied_usd'] + df['total_repaid_usd'])

# Binary flags
df['is_inactive'] = (df['days_since_last_txn'] > 180).astype(int)
df['is_frequent_liquidated'] = (df['liquidation_count'] > 0).astype(int)

# Normalize features to [0, 1]
df['borrow_to_supply_score'] = np.clip(df['borrow_to_supply_ratio'], 0, 1)
df['repayment_score'] = 1 - np.clip(df['repayment_ratio'], 0, 1)
df['utilization_score'] = np.clip(df['utilization_rate'], 0, 1)

# Compute final risk score (weighted)
df['score'] = 1000 * (
    0.25 * df['borrow_to_supply_score'] +
    0.25 * df['utilization_score'] +
    0.2  * df['repayment_score'] +
    0.15 * df['is_frequent_liquidated'] +
    0.15 * df['is_inactive']
)

# Round and clip scores
df['score'] = df['score'].clip(0, 1000).round().astype(int)

# Save output
df[['wallet_id', 'score']].to_csv("wallet_risk_scores.csv", index=False)
